In [1]:
import os
import math
from PIL import Image
import pandas as pd
import numpy as np
#import matplotlib.pyplot as plt


from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.layers import Activation, Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization

In [2]:
IDToIndex = {}
indexToID = {}
IDtonum = {}
IDtoimg={}

index = -3
for dirname, _, filenames in os.walk('./insect/database'):
    if(index > -1):
        specie = dirname.rsplit('/', 1)[1] 
        IDToIndex[specie] = index
        indexToID[index] = specie
        IDtonum[specie]=len(filenames)
        IDtoimg[specie]=filenames
    index = index + 1
#Creating our first dataframe of Insect IDs and picture file names
predf = []
for specie, imgs in IDtoimg.items():
    for img in imgs:
        predf.append({'ID': specie, 'imgfile': img})
df1 = pd.DataFrame(predf)
print(df1)

            ID        imgfile
0      4470801  d015s0007.jpg
1      4470801  d015s0103.jpg
2      4470801  d015s0023.jpg
3      4470801  d015s0065.jpg
4      4470801  d015s0118.jpg
5      4470801  d015s0017.jpg
6      4470801  d015s0124.jpg
7      4470801  d015s0049.jpg
8      4470801  d015s0052.jpg
9      4470801  d015s0080.jpg
10     4470801  d015s0073.jpg
11     4470801  d015s0011.jpg
12     4470801  d015s0055.jpg
13     4470801  d015s0081.jpg
14     4470801  d015s0108.jpg
15     4470801  d015s0013.jpg
16     4470801  d015s0056.jpg
17     4470801  d015s0020.jpg
18     4470801  d015s0106.jpg
19     4470801  d015s0012.jpg
20     4470801  d015s0072.jpg
21     4470801  d015s0006.jpg
22     4470801  d015s0116.jpg
23     4470801  d015s0061.jpg
24     4470801  d015s0123.jpg
25     4470801  d015s0120.jpg
26     4470801  d015s0067.jpg
27     4470801  d015s0019.jpg
28     4470801  d015s0038.jpg
29     4470801  d015s0090.jpg
...        ...            ...
62751  1037319  d060s0056.jpg
62752  103

In [3]:
import collections 
idToNum_df = pd.DataFrame(IDtonum.items())

In [4]:
i = 0
top10 = []
for x in sorted(IDtonum, key=IDtonum.get, reverse=True):
    if i < 10:
        top10.append([int(x), IDtonum[x]])
        i+=1
print(top10)

top10df = pd.DataFrame(top10, columns=['key', 'quantity'])
print(top10df)

[[9364935, 888], [1035931, 861], [7508714, 818], [4475140, 719], [1036216, 686], [5755079, 651], [4474169, 642], [1035185, 599], [1035195, 570], [1035864, 566]]
       key  quantity
0  9364935       888
1  1035931       861
2  7508714       818
3  4475140       719
4  1036216       686
5  5755079       651
6  4474169       642
7  1035185       599
8  1035195       570
9  1035864       566


In [12]:
data = {
    'key': [1036216, 7508714, 1035931, 4475140, 1035864, 1035185, 4474169, 5755079, 1035195, 9364935],
    'family': ['Carabidae'] * 10,
    # 'genus': ['Nebria', 'Bembidion', 'Pterostichus', 'Pterostichus', 'Bembidion', 'Elaphrus', 'Ophonus', 
    #           'Pterostichus', 'Harpalus', 'Pterostichus', 'Bembidion', 'Amara', 'Calathus', 'Bembidion', 
    #           'Paranchus', 'Anchomenus', 'Agonum', 'Amara', 'Notiophilus', 'Acupalpus'],
    # 'species': ['Nebria brevicollis', 'Bembidion lampros', 'Pterostichus nigrita', 'Pterostichus strenuus', 
    #             'Bembidion tetracolum', 'Elaphrus riparius', 'Ophonus rufibarbis', 'Pterostichus madidus', 
    #             'Harpalus serripes', 'Pterostichus diligens', 'Bembidion dentellum', 'Amara aenea', 
    #             'Calathus melanocephalus', 'Bembidion guttula', 'Paranchus albipes', 'Anchomenus dorsalis', 
    #             'Agonum fuliginosum', 'Amara familiaris', 'Notiophilus biguttatus', 'Acupalpus dubius']
}
species10 = pd.DataFrame(data)

In [13]:
species10_q = species10.merge(top10df, on='key', how='left')
print(species10_q)

      family      key  quantity
0  Carabidae  1036216       686
1  Carabidae  7508714       818
2  Carabidae  1035931       861
3  Carabidae  4475140       719
4  Carabidae  1035864       566
5  Carabidae  1035185       599
6  Carabidae  4474169       642
7  Carabidae  5755079       651
8  Carabidae  1035195       570
9  Carabidae  9364935       888


In [14]:
num_per_spec = species10_q.quantity.tolist()
print(num_per_spec)
id_to_label = {}
id_spec = species10_q.key.tolist()
for idx, id_ in enumerate(id_spec):
    id_to_label[id_] = idx
print(id_to_label)

[686, 818, 861, 719, 566, 599, 642, 651, 570, 888]
{1035195: 8, 4475140: 3, 9364935: 9, 1035185: 5, 7508714: 1, 5755079: 7, 1035864: 4, 1036216: 0, 4474169: 6, 1035931: 2}


In [15]:
top10_str = []
for x in top10df['key']:
    top10_str.append(str(x))
    
df2 = df1[df1['ID'].isin(top10_str)].reset_index(drop=True)
print(df2)

           ID        imgfile
0     1036216  d068s0002.jpg
1     1036216  d068s0502.jpg
2     1036216  d068s0458.jpg
3     1036216  d068s0145.jpg
4     1036216  d068s0003.jpg
5     1036216  d068s0609.jpg
6     1036216  d068s0228.jpg
7     1036216  d068s0414.jpg
8     1036216  d068s0064.jpg
9     1036216  d068s0604.jpg
10    1036216  d068s0467.jpg
11    1036216  d068s0565.jpg
12    1036216  d068s0133.jpg
13    1036216  d068s0175.jpg
14    1036216  d068s0299.jpg
15    1036216  d068s0132.jpg
16    1036216  d068s0112.jpg
17    1036216  d068s0533.jpg
18    1036216  d068s0583.jpg
19    1036216  d068s0333.jpg
20    1036216  d068s0075.jpg
21    1036216  d068s0078.jpg
22    1036216  d068s0262.jpg
23    1036216  d068s0503.jpg
24    1036216  d068s0260.jpg
25    1036216  d068s0014.jpg
26    1036216  d068s0679.jpg
27    1036216  d068s0085.jpg
28    1036216  d068s0544.jpg
29    1036216  d068s0452.jpg
...       ...            ...
6970  4474169  d140s0223.jpg
6971  4474169  d141s0105.jpg
6972  4474169 

In [16]:
import os
import numpy as np
import cv2
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

In [17]:
data_dir = "./insect/database/"
classes = os.listdir(data_dir)
class_to_index = {cls: i for i, cls in enumerate(classes)}
index_to_class = {i: cls for i, cls in enumerate(classes)}

In [18]:
images = []
labels = []

img_height = 28
img_width = 28

for cls in classes:
    if cls in top10_str:  # 상위 20종만 처리, 종의 수에 따라 조정 가능
        cls_dir = os.path.join(data_dir, cls)
        for img_file in os.listdir(cls_dir):
            img_path = os.path.join(cls_dir, img_file)
            
            # 이미지를 흑백 모드로 읽음
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            
            # 이미지를 28x28 크기로 리사이즈
            img = cv2.resize(img, (img_height, img_width))
            img = img / 255.0
            images.append(img)
            label = np.zeros(10)
            label[id_to_label[int(cls)]] = 1
            labels.append(label)            
images = np.array(images)
labels = np.array(labels)

images = np.expand_dims(images, axis=-1)
print(images.shape)
print(labels.shape)

(7000, 28, 28, 1)
(7000, 10)


In [19]:
# Step 2: Data Splitting
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

In [93]:
# unique = list(set(y_train))
# value_to_index = {value:idx for idx, value in enumerate(unique)}
# one_hot_encoded = np.zeros((len(y_train), len(unique_values)))
# for i, value

SyntaxError: invalid syntax (<ipython-input-93-b6968a96bc9a>, line 4)

In [20]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [21]:
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(
    monitor='val_accuracy',  # Metric to monitor
    patience=10,         # Number of epochs with no improvement after which training will be stopped
    verbose=1,           # To log the number of epochs after which training was stopped
    restore_best_weights=True  # Restores model weights from the epoch with the minimum validation loss
)

In [22]:
from tensorflow.keras.callbacks import ModelCheckpoint

model_checkpoint = ModelCheckpoint(
    'bug_model1.keras',        # Path where to save the model
    monitor='val_accuracy', # Metric to monitor
    save_best_only=True,    # Only save a model if `val_accuracy` has improved
    verbose=1               # Log when a new best model is saved
)

In [25]:
num_classes = 10 # change based on species num
from tensorflow.keras.optimizers import Adam

model = Sequential()

# Conv layer 1
model.add(Conv2D(16, (3, 3), activation='relu', input_shape=(img_height, img_width, 1), name="conv1"))
model.add(MaxPooling2D((2, 2)))

# Conv layer 2
model.add(Conv2D(16, (5, 5), activation='relu', name="conv2"))
model.add(MaxPooling2D((2, 2)))

# Flatten and Dense layers
model.add(Flatten())
model.add(Dense(32, activation='relu', name="fc1"))

# Output layer
model.add(Dense(num_classes, activation='softmax', name = "fc2"))

# Compile
model.compile(Adam(learning_rate=0.005),
              loss='categorical_crossentropy',
              metrics=['accuracy'])
# 모델 구조 확인
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1 (Conv2D)               (None, 26, 26, 16)        160       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 13, 13, 16)        0         
_________________________________________________________________
conv2 (Conv2D)               (None, 9, 9, 16)          6416      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 4, 4, 16)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 256)               0         
_________________________________________________________________
fc1 (Dense)                  (None, 32)                8224      
_________________________________________________________________
fc2 (Dense)                  (None, 10)               

In [26]:
history = model.fit(
    X_train, y_train,
    epochs=50,
    batch_size=32,
    validation_data=(X_test, y_test),
    callbacks=[early_stopping, model_checkpoint]  
)

Train on 5600 samples, validate on 1400 samples
Epoch 1/50
5600/5600 [==============================] - 6s 1ms/sample - loss: 2.1377 - acc: 0.1825 - val_loss: 2.0228 - val_acc: 0.2136
Epoch 2/50
5600/5600 [==============================] - 5s 901us/sample - loss: 1.9119 - acc: 0.2918 - val_loss: 1.8661 - val_acc: 0.3236
Epoch 3/50
5600/5600 [==============================] - 5s 854us/sample - loss: 1.7568 - acc: 0.3557 - val_loss: 1.7140 - val_acc: 0.3750
Epoch 4/50
5600/5600 [==============================] - 7s 1ms/sample - loss: 1.6496 - acc: 0.4029 - val_loss: 1.6672 - val_acc: 0.3664
Epoch 5/50
5600/5600 [==============================] - 4s 724us/sample - loss: 1.5958 - acc: 0.4182 - val_loss: 1.6001 - val_acc: 0.4257
Epoch 6/50
5600/5600 [==============================] - 4s 680us/sample - loss: 1.5242 - acc: 0.4505 - val_loss: 1.5941 - val_acc: 0.4186
Epoch 7/50
5600/5600 [==============================] - 5s 930us/sample - loss: 1.4869 - acc: 0.4586 - val_loss: 1.5483 - val_ac

In [27]:
import tensorflow as tf
print(tf.__version__)

1.15.0


In [28]:
import pickle

In [31]:
# Save weights and biases function
def save_layer_params(layer_name, layer):
    layer_name = './params/'+layer_name
    # 가중치와 편향을 가져옴
    weights, biases = layer.get_weights()
    print(weights.shape)
    print(layer_name)
    # 가중치 저장
    with open(layer_name + '_w.param', 'wb') as f_w:
        pickle.dump(weights, f_w)
    
    # 편향 저장
    with open(layer_name + '_b.param', 'wb') as f_b:
        pickle.dump(biases, f_b)

In [32]:
# 모델의 각 레이어별로 가중치와 편향 저장
for layer in model.layers:
    if len(layer.get_weights()) > 0:  # 가중치가 있는 레이어만 저장
        print("hello")
        save_layer_params(layer.name, layer)

hello
(3, 3, 1, 16)
./params/conv1
hello
(5, 5, 16, 16)
./params/conv2
hello
(256, 32)
./params/fc1
hello
(32, 10)
./params/fc2


In [33]:
def save_first_data(df, save_path='./input.param'):
    first_data = df.iloc[0]
    with open(save_path, 'wb') as f:
        pickle.dump(first_data, f)

In [38]:
temp = X_train[0].reshape(28, 28)
with open('./input.param', 'wb') as f: 
    pickle.dump(temp, f)

In [39]:
print(y_train[0])

[0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]


In [41]:
model.predict(X_train[0].reshape(1,28,28,1))

array([[2.1648081e-04, 2.5093153e-01, 1.5717106e-01, 1.5328967e-03,
        1.0317753e-04, 2.4505269e-05, 1.4516088e-01, 4.4471222e-01,
        7.1555536e-05, 7.5757678e-05]], dtype=float32)